In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from cot.data.data_processing import Parity, BinaryCopy, Copy
from cot.config import RAW_DIR

import logging
logging.basicConfig(level=logging.INFO)

## DataLoader

In [3]:
Problem = BinaryCopy

In [4]:
rng = np.random.default_rng()
seq_length = 20
max_nb_data_per_len = 1000
random = False
split_probas_by_len =  [1, 1, 1, 1, .9, .8, .9, .3, .2]
probas_by_len = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1]).astype(float)
probas_by_len /= probas_by_len.sum()

lengths = list(np.arange(len(split_probas_by_len)) + 1)

In [5]:
if Problem.prefix == 'copy':
    Problem(vocab_size=20)

Problem.generate_datafiles(max_nb_data_per_len, split_probas_by_len, rng)

trainset = Problem()
trainset.set_as_trainset(lengths, probas_by_len)

testset = Problem()
testset.set_as_testset(lengths)

INFO:cot.data.data_processing:Sequences of length 1 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (2/2 split).
INFO:cot.data.data_processing:Sequences of length 2 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (4/4 split).
INFO:cot.data.data_processing:Sequences of length 3 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (8/8 split).
INFO:cot.data.data_processing:Sequences of length 4 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (16/16 split).
INFO:cot.data.data_processing:Sequences of length 5 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (28/32 split).
INFO:cot.data.data_processing:Sequences of length 6 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (51/64 split).
INFO:cot.data.data_processing:Sequences of length 7 done. Saved in /home/vivc/Code/llm/Compositionality/data/raw/binary_copy (115/128 split).
INFO:cot.data.data_proce

In [8]:
loader = DataLoader(trainset, batch_size=32, sampler=trainset.sampler)

## Training loop: TODO

In [9]:
for batch in loader:
    print(batch)
    break

tensor([[ 1,  0,  0,  2,  1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 0,  1,  0,  1,  1,  0,  0,  2,  0,  1,  0,  1,  1,  0,  0, -1, -1, -1,
         -1, -1],
        [ 0,  0,  0,  1,  1,  1,  1,  1,  1,  2,  0,  0,  0,  1,  1,  1,  1,  1,
          1, -1],
        [ 0,  1,  0,  2,  0,  1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 1,  1,  1,  0,  1,  0,  2,  1,  1,  1,  0,  1,  0, -1, -1, -1, -1, -1,
         -1, -1],
        [ 1,  0,  1,  1,  2,  1,  0,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 1,  0,  1,  2,  1,  0,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 1,  2,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 1,  2,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [ 0,  0,  0,  1,  0,  0,  0,  2,  0,  0,  0,  1,  0,  0,  0, -1, -1, -1,
         -1, -1],
        [ 